In [11]:
import numpy as np
import nltk
import pickle
import json

from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

nltk.download('punkt')
nltk.download('wordnet')
lemmatizer=WordNetLemmatizer()
intents=json.loads(open('intents.json').read())
words=[]
classes=[]
documents=[]
ignore_letters=['!','@','?','.','/','%','$']
for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list=nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list,intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))
print(words)

print(classes)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
["'s", ',', 'Adverse', 'Awesome', 'Blood', 'Bye', 'Find', 'Give', 'Good', 'Goodbye', 'Hello', 'Hey', 'Hi', 'Hola', 'Hospital', 'How', 'I', 'ID', 'Is', 'List', 'Load', 'Locate', 'Looking', 'Lookup', 'Nice', 'Open', 'Search', 'Searching', 'See', 'Show', 'Task', 'Thank', 'Thanks', 'That', 'Till', 'What', 'Which', 'a', 'adverse', 'all', 'anyone', 'are', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'for', 'have', 'help', 'helpful', 'helping', 'history', 'hospital', 'is', 'later', 'list', 'log', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'of', 'offered', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'suitable', 'sup

In [14]:
import random

In [36]:
training=[]
output_empty=[0]*len(classes)

for document in documents:
  bag=[]
  word_patterns=document[0]
  word_patterns=[lemmatizer.lemmatize(word) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)
  output_row=list(output_empty)
  output_row[classes.index(document[1])]=1
  training.append((bag,output_row))
random.shuffle(training)
training=np.array(training)
train_x=list(training[:,0])
train_y=list(training[:,1])

model = Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


Epoch 1/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1342 - accuracy: 0.1433
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1669 - accuracy: 0.0768
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0166 - accuracy: 0.3357
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9926 - accuracy: 0.3117
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 1.7860 - accuracy: 0.4873
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 1.7716 - accuracy: 0.4834
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.5712 - accuracy: 0.6180
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.4323 - accuracy: 0.6955
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.1164 - accuracy: 0.7075
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.1242 - accuracy: 0.7552
Epoch 11/

In [37]:
from tensorflow.keras.models import load_model
lemmatizer=WordNetLemmatizer()
intents=json.loads(open('intents.json').read())
words=pickle.load(open('words.pkl','rb'))
classes=pickle.load(open('classes.pkl','rb'))
model=load_model('chatbot_model.h5')

def clean_up_sentence(sentence):
  sentence_words=nltk.word_tokenize(sentence)
  sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words
def bag_of_words(sentence):
  sentence_words=clean_up_sentence(sentence)
  bag=[0]*len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word==w:
        bag[i]=1
  return np.array(bag)

def predict_class(sentence):
  bow=bag_of_words(sentence)
  res=model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD=0.245
  result=[[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]

  result.sort(key=lambda x:x[1],reverse=True)
  return_list=[]
  for r in result:
    return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
  return return_list


In [38]:
def get_response(intent_list,intent_json):
  tag=intent_list[0]['intent']
  list_of_intents = intent_json['intents']
  for i in list_of_intents:
    if i['tag']==tag:
      result=random.choice(i['responses'])
      break
  return result

In [ ]:
while True:
  message=input("Inp")
  ints=predict_class(message)
  print(ints)
  res=get_response(ints, intents)
  print(res)

Inphello
[{'intent': 'greeting', 'probability': '0.8012707'}]
Hi there, how can I help?
Inphelp me
[{'intent': 'options', 'probability': '0.9622064'}]
I can guide you through Adverse drug reaction list, Blood pressure tracking, Hospitals and Pharmacies
Inphowdy
[{'intent': 'greeting', 'probability': '0.8012707'}]
Good to see you again
Inpsearch
[{'intent': 'greeting', 'probability': '0.44216028'}]
Hello, thanks for asking
Inpsearch me a hospital
[{'intent': 'hospital_search', 'probability': '0.9957735'}]
Please provide hospital name or location
Inpfuck you
[{'intent': 'goodbye', 'probability': '0.43600467'}, {'intent': 'thanks', 'probability': '0.2871508'}]
See you!
Inpkaise ho
[{'intent': 'greeting', 'probability': '0.8012707'}]
Hi there, how can I help?
Inpgaana sunao
[{'intent': 'greeting', 'probability': '0.8012707'}]
Good to see you again


In [33]:
print(train_y)

[[0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 